## Filter Methods - Correlation 

In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import gc
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split

### load dataset

In [ ]:
# load dataset and features from previus method
features = np.load('featuresFromBasicMethods.npy').tolist()
data = pd.read_pickle('../../data/features/features.pkl').loc[:,features].sample(frac=0.35).fillna(-9999)

ddata = dd.from_pandas(data, npartitions=8)

ddata.shape

In [ ]:
ddata.head()

In [ ]:
# In practice, feature selection should be done after data pre-processing,
# so ideally, all the categorical variables are encoded into numbers,
# and then you can assess whether they are correlated with other features

# here for simplicity I will use only numerical variables
# select numerical columns:

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(ddata.select_dtypes(include=numerics).columns)
ddata = ddata[numerical_vars]
ddata.shape

### Brute force approach

In [ ]:
# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything else
# without any other insight.

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr().compute()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(ddata, 0.88)
len(set(corr_features))

In [ ]:
feat2keep = [c for c in ddata.columns if c not in corr_features]

In [ ]:
dddata = ddata.drop(labels=list(corr_features),axis=1)

### plot matrix correlation

In [ ]:
# visualise correlated features
# I will build the correlation matrix, which examines the 
# correlation of all features (for all possible feature combinations)
# and then visualise the correlation matrix using seaborn
corr_matrix = dddata.corr().compute()

fig, ax = plt.subplots()
fig.set_size_inches(11,11)
sns.heatmap(corr_matrix)

### save features

In [ ]:
np.save('../features/featuresFromCorrelation.npy',feat2keep)

## Second approach

The second approach looks to identify groups of highly correlated features. And then, we can make further investigation within these groups to decide which feature we keep and which one we remove.

In [ ]:
ddata = dd.from_pandas(data, npartitions=8)


In [ ]:
# In practice, feature selection should be done after data pre-processing,
# so ideally, all the categorical variables are encoded into numbers,
# and then you can assess whether they are correlated with other features

# here for simplicity I will use only numerical variables
# select numerical columns:

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(ddata.select_dtypes(include=numerics).columns)
ddata = ddata[numerical_vars]
ddata.shape

In [ ]:
ddata.head()

In [ ]:
# build a dataframe with the correlation between features
# remember that the absolute value of the correlation
# coefficient is important and not the sign

corrmat = dddata.corr().compute()
corrmat = corrmat.abs().unstack() # absolute value of corr coef
corrmat = corrmat.sort_values(ascending=False)
corrmat = corrmat[corrmat >= 0.8]
corrmat = corrmat[corrmat < 1]
corrmat = pd.DataFrame(corrmat).reset_index()
corrmat.columns = ['feature1', 'feature2', 'corr']
corrmat.head()

In [ ]:
# find groups of correlated features

grouped_feature_ls = []
correlated_groups = []

for feature in corrmat.feature1.unique():
    if feature not in grouped_feature_ls:

        # find all features correlated to a single feature
        correlated_block = corrmat[corrmat.feature1 == feature]
        grouped_feature_ls = grouped_feature_ls + list(
            correlated_block.feature2.unique()) + [feature]

        # append the block of features to the list
        correlated_groups.append(correlated_block)

print('found {} correlated groups'.format(len(correlated_groups)))
print('out of {} total features'.format(X_train.shape[1]))

In [ ]:
# now we can visualise each group. We see that some groups contain
# only 2 correlated features, some other groups present several features 
# that are correlated among themselves.

for group in correlated_groups:
    print(group)
    print()

In [ ]:
# we can now investigate further features within one group.
# let's for example select group 3

group = correlated_groups[2]
group